In [24]:
import pymongo
import re
import mysql.connector
from pymongo import MongoClient
import datetime as dt
import mysql.connector
from pymongo import MongoClient
from config import MYSQL_CONFIG,FILE_PATH,MONGO_URL

## Extraction

In [5]:
def email_date_extract(file_path):
    email = re.compile(r"From: (\S+@\S+)")
    date= re.compile(r"^Date: (.*)$")
    extracted_data = []
    current_email, current_date = None, None
    with open(f'{file_path}','r',encoding="UTF 8") as file:
        for line in file:
            email_matching=email.search(line)
            date_matching=date.search(line)
            if email_matching:
                current_email = email_matching.group(1)
            if date_matching: 
                current_date = date_matching.group(1)
            if current_email and current_date:
                extracted_data.append((current_email, current_date))
                current_email, current_date = None, None
    return extracted_data

In [12]:
result=email_date_extract(FILE_PATH)

In [17]:
print(result[1])

('louis@media.berkeley.edu', 'Fri, 4 Jan 2008 18:08:57 -0500')


## Transformation

In [6]:
import datetime as dt

def transformation(data):
    transformed=[]

    for email,date in data:
        formatted_date=dt.datetime.strptime('Fri, 4 Jan 2008 18:08:57 -0500',"%a, %d %b %Y %H:%M:%S %z") \
        .strftime("%Y-%m-%d %H:%M:%S")
        transformed.append((email,formatted_date))

    return transformed

In [14]:
t_result=transformation(result)

In [16]:
print(t_result[1])

('louis@media.berkeley.edu', '2008-01-04 18:08:57')


## Load to mongo

In [25]:
def load_mongo(data,db_name="guvi_db", collection_name="user_history"):
    formatted_data = [{"email": email, "date": date} for email, date in data]
    client = MongoClient(MONGO_URL) 
    db = client[db_name]
    collection = db[collection_name]
    collection.insert_many(formatted_data)

In [26]:
load_mongo(t_result)

In [30]:
def load_mysql():
    myclient = pymongo.MongoClient(MONGO_URL)
    mydb = myclient["guvi_db"]
    mycollection=mydb['user_history']

    data = list(mycollection.find({}, {"_id": 0, "email": 1, "date": 1}))


    cnx= mysql.connector.connect(**MYSQL_CONFIG)
    mycursor = cnx.cursor()
    mycursor.execute("DROP TABLE IF EXISTS user_history")
    mycursor.execute("""CREATE TABLE user_history ( Personid int AUTO_INCREMENT primary key,
    email VARCHAR(255), date timestamp)""")

    insert_query="Insert into user_history (email,date) values(%s,%s) "
    mycursor.executemany(insert_query, [(d["email"], d["date"]) for d in data])

    cnx.commit()
    mycursor.close()
    cnx.close()

## Queries

In [51]:
queries = {
        "List all unique email address": "SELECT DISTINCT email FROM user_history;",
        "Count emails per day": "SELECT DATE(date) AS day, COUNT(email) FROM user_history GROUP BY day;",
        "First and last email date per email": """
         SELECT email, MIN(date) as first_email, MAX(date) as last_email
         FROM user_history GROUP BY email;
        """,
        "Count emails per domain" : """
         SELECT SUBSTRING_INDEX(email, '@', -1) AS domain, COUNT(email)
         FROM user_history GROUP BY domain; 
        """
    }

def execute_queries():
    cnx = mysql.connector.connect(**MYSQL_CONFIG)
    mycursor = cnx.cursor(buffered=True)

    for title, query in queries.items():
        print(f"\n{title}:")
        mycursor.execute(query)
        rows = mycursor.fetchall()
        
        if rows:
            for row in rows:
                print(row)
        else:
            print("No results found.")
        
        # Ensure the results are consumed before running another query
        mycursor.nextset()

    # Close the cursor and connection
    mycursor.close()
    cnx.close()


In [52]:
execute_queries()


List all unique email address:
('stephen.marquard@uct.ac.za',)
('louis@media.berkeley.edu',)
('zqian@umich.edu',)
('rjlowe@iupui.edu',)
('cwen@iupui.edu',)
('gsilver@umich.edu',)
('wagnermr@iupui.edu',)
('antranig@caret.cam.ac.uk',)
('gopal.ramasammycook@gmail.com',)
('david.horwitz@uct.ac.za',)
('ray@media.berkeley.edu',)
('mmmay@indiana.edu',)
('stuart.freeman@et.gatech.edu',)
('tnguyen@iupui.edu',)
('chmaurer@iupui.edu',)
('aaronz@vt.edu',)
('ian@caret.cam.ac.uk',)
('csev@umich.edu',)
('jimeng@umich.edu',)
('josrodri@iupui.edu',)
('knoop@umich.edu',)
('bkirschn@umich.edu',)
('dlhaines@umich.edu',)
('hu2@iupui.edu',)
('sgithens@caret.cam.ac.uk',)
('arwhyte@umich.edu',)
('gbhatnag@umich.edu',)
('gjthomas@iupui.edu',)
('a.fish@lancaster.ac.uk',)
('ajpoland@iupui.edu',)
('lance@indiana.edu',)
('ssmail@indiana.edu',)
('jlrenfro@ucdavis.edu',)
('nuno@ufp.pt',)
('zach.thomas@txstate.edu',)
('ktsao@stanford.edu',)
('ostermmg@whitman.edu',)
('john.ellis@rsmart.com',)
('jleasia@umich.edu',)
